# Street types in Gothenburg

Swedish streets have many different endings, identifying the type of street. The most common ones are "gata/gatan" (street), "väg/vägen" (road or way), "leden" (motorway) and "torg/torget" (square). 

In this analysis, I wanted to visualize if in Gothenburg there is a pattern of street types, or if they just randomly distributed around the city. 

The steps to achieve this visualization were the following:

1. Get the geographical information (coordinates) and street names data
2. Categorize the different street types
3. Visualize the street types on a map

I'll walk you through the different steps in detail. 
The tools I used for this visualization were:
- [OpenStreetMap](https://www.openstreetmap.org/)
- [Overpass Turbo API](https://overpass-turbo.eu/)
- [Mapshaper](https://mapshaper.org/)
- [Python](https://www.python.org/) with the [pandas](https://pandas.pydata.org/) and [numpy](https://numpy.org/) libraries
- [Mapbox](https://studio.mapbox.com)


## 1. Get the geographical information (coordinates) and street names data

To extract information from OpenStreetMap, I used the Overpass Turbo API. Since I wanted all possible streets I used a wildcard (*) in my query. This selects “residential”, “primary”, “secondary”, “tertiary”, etc., roads. Search area was set to “Göteborg” (since “Gothenburg” leads to a town in Nebraska).

`// fetch area “Göteborg” to search in
{{geocodeArea:Göteborg}}->.searchArea;
Highway = * `

This query took a long time, so I had to increase the timeout time for it. The resulting geojson file was quite big (>80 MB).

For the next step I wanted a csv instead of a geojson, and an easy way to convert one to the other is using Mapshaper: you just import the geojson file of choice, then export it as a csv.

Mapshaper had no problem importing the large geojson.

## 2. Categorize the different street types

So what I wanted to do next is load the csv with all the streets information into Python, then slim down the csv to a more manageable size (in the full form it had >60 000 rows and >250 columns) and then run some kind of function on it that would check the street name and decide if it was a vägen, gatan, gränd, backen, leden, etc. 

For this I would need the libraries pandas and numpy, so that's what I imported first. Then I read in the csv.

In [1]:
import pandas as pd
import numpy as np

#import the csv with all GBG highways
GBG_all = pd.read_csv("files/GBG all highways simplified.csv")
#Look at the first 5 rows
GBG_all.head()

/Users/silfa/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,4,10,11,12,13,16,17,19,22,23,24,26,27,28,29,30,31,32,33,35,37,38,39,40,42,43,44,45,46,47,50,51,52,54,55,58,59,60,61,63,64,66,67,68,69,70,71,72,73,74,75,77,78,79,80,81,82,83,85,86,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,116,117,119,120,121,122,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,144,145,146,147,148,149,151,153,154,155,158,162,163,164,165,166,167,168,169,171,172,173,174,175,176,177,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,198,199,200,201,202,203,204,205,206,207,208,210,211,212,216,217,218,219,220,221,222,223,224,226,227,228,229,230,231,233,234,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.bod

,@id,highway,int_ref,lanes,maxspeed,name,oneway,ref,surface,FID,...,ford,asphalt,wikimedia_commons,passenger_information_display,parking:condition:left,parking:lane:left,direction,oneway:foot,website,@relations
0,way/4040302,motorway,E 06,2.0,80,Kungälvsleden,yes,E 6,asphalt,way/4040302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,way/4040303,motorway,E 06,3.0,100,NaN,yes,E 6,asphalt,way/4040303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,way/4040436,motorway_link,NaN,1.0,80,NaN,yes,NaN,asphalt,way/4040436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,way/4040439,motorway,E 20,1.0,70,NaN,yes,E 20,asphalt,way/4040439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,way/4040441,motorway,E 20,3.0,70,NaN,yes,E 20,asphalt,way/4040441,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As mentioned, the csv is huge and contains a lot of rows that are other geospatial information, but not actual streets (see rows with index 1-4 above). Also, for most streets there are several street segements that all fill a separate row, and to get rid of that redundancy I first created a new list with only the street names (i.e. the column "name") that are unique (i.e. only one entry per unique street name). In my analysis I did not care about the other information that came in the geojson/csv, like "surface" or "oneway". 

This list of unique street names was then turned into a dataframe. Subsequently, I also removed the one entry that was NaN.

In [2]:
#Select only the name column and make the list unique
GBG_unique_list = GBG_all.name.unique().tolist() #Make a list with the unique values
GBG_unique_dict = {"name": GBG_unique_list} #Turn the list into a dictionary
GBG_unique_df = pd.DataFrame(GBG_unique_dict) #Turn the dictionary into a dataframe


#Show the first 5 rows of the new csv
GBG_unique_df.head()

,name
0,Kungälvsleden
1,NaN
2,Boråsleden
3,Gråbovägen
4,Hjällbovägen


In [3]:
#Remove the NaN value
GBG_unique_df = GBG_unique_df.dropna()
GBG_unique_df.head()

,name
0,Kungälvsleden
2,Boråsleden
3,Gråbovägen
4,Hjällbovägen
5,Alingsåsleden


Conventiently, the street types are usually at the end of the street name, so I could make use of the pandas function `.str.endswith`. I tried that out with "vägen", and it worked.

In [4]:
only_vagen = GBG_unique_df[GBG_unique_df["name"].str.endswith("vägen")]
only_vagen.head()

,name
3,Gråbovägen
4,Hjällbovägen
29,Skattegårdsvägen
33,Näsetvägen
55,Gamlestadsvägen


What I wanted to do next is to create a certain value in a new column based on a condition. For this, the numpy function [numpy.select()](https://numpy.org/doc/stable/reference/generated/numpy.select.html) seemed quite suitable.

`numpy.select(condlist, choicelist, default=0)`

It takes a **list of conditions** and a **list of choices** (they need to have the same length). The list of conditions determines which element in the choice list is taken as an output. 

The **default** value is the output value when all other conditions evaluate to `False`.

So first, with the `str.endswith` function I created a list of conditions that checks whether a certain street name ends in a different endings. I only used the most common ones here, even though there are a dozen others. 

Then I made a list with the values I wanted to assign, and you can see some redudancy there, because I had to account for different ways of spelling one and the same thing (gata/gatan/Gata, etc.). 

In [5]:
#Create a list of our conditions
conditions = [
    (GBG_unique_df["name"].str.endswith("vägen")==True),
    (GBG_unique_df["name"].str.endswith("väg")==True),
    (GBG_unique_df["name"].str.endswith("gatan")==True),
    (GBG_unique_df["name"].str.endswith("gata")==True),
    (GBG_unique_df["name"].str.endswith("Gata")==True),
    (GBG_unique_df["name"].str.endswith("backen")==True),
    (GBG_unique_df["name"].str.endswith("gränd")==True),
    (GBG_unique_df["name"].str.endswith("torg")==True),
    (GBG_unique_df["name"].str.endswith("torget")==True),
    (GBG_unique_df["name"].str.endswith("motet")==True),
    (GBG_unique_df["name"].str.endswith("leden")==True),
]


#Create a list of the values we want to assign for each condition
values = ["vägen", "vägen", "gatan", "gatan", "gatan", "backen", "gränd", "torget", "torget", "motet", "leden"]


In [6]:
#Create a new column and use np.select to assign values to it using our list as arguments
GBG_unique_df["street_type"] = np.select(conditions, values, "other")

To check if it worked, I looked at the first 5 lines, and everything seemed fine:

In [7]:
GBG_unique_df.head()

,name,street_type
0,Kungälvsleden,leden
2,Boråsleden,leden
3,Gråbovägen,vägen
4,Hjällbovägen,vägen
5,Alingsåsleden,leden


Now all that was left to do is to turn this dataframe into a csv again.

In [8]:
#Turn the dataframe into a csv
GBG_unique_df.to_csv("files/GBG street types.csv")

## 3. Visualize the street types on a map

### Joining map and data

First, it was necessary to join the new, slim csv with the names and the street types with the original geojson that contains all the important geospatial information. 

In Mapshaper this was done very easily by just uploading both of these files (map, data) and then joining them by a common column header (i.e. "name") in the Mapshaper console by typeing: 

`join data keys=name,name`

Mapshaper is case-sensitive and kind of picky, it's important to not introduce any additional spaces in that command.

I checked if the map was properly joined by selecting the arrow icon and "select feature", then hovering over the streets and seeing if the new "street_type" category shows up in the information. This was the case, so we could now export that new geojson by clicking "export" and selecting only the map layer. 

### Creating the map

In Mapbox, I went to Tilesets > New Tileset and uploaded the newly created geojson.
Once that was done, I went to Styles > New Style and chose a template I liked, then clicked "Customize". 

First, I navigated to my location of choice, i.e. "Gothenburg". The streets and other data were not yet visible because I needed to add them first. 
In the "Components" tab, I clicked the large + symbol, then selected Data Visualization. 
As a source, I selected the tileset I created earlier.
Then I clicked on "Select visualization type" and then the recommended Data-driven lines.

By default, some category is already selected and coloured, but I removed that pre-selected data by clicking on the bin symbol next to it. 
Then I scrolled down to Color and clicked on the three dots, and selected the data field "street_type". 

Up to 7 different variables in this field can be visualised. (If you want more than that, you'll have to do it manually with layers, but I chose to work with only 7).

I adjusted the colours to be red for all street types "gatan" and blue for all street types "vägen". All other street types I gave the same shade of green. 

I also adjusted the line width by selecting it in the Size field above Color. 

There is a ton of other stuff that can be customized, like data labels. But I kept it simply in this visualization, because I just wanted to get a bird's eye view of the colour distribution in the city. 

Once I was happy with the map, I clicked "Publish" in the right top corner. Once that is done, you can click on "Share" and get a link to the interactive map that lets you zoom in and out and scroll around.

My interactive map can be found here: [GBG street types](https://api.mapbox.com/styles/v1/silfaz/cl0xvwbhu000v14lp723p61io.html?title=view&access_token=pk.eyJ1Ijoic2lsZmF6IiwiYSI6ImNrenRuY2NrdTEydzEybnBraGszaWpuOHUifQ.f8NFzJ-yyaUfTpP6Vn3maA&zoomwheel=true&fresh=true#11.06/57.7169/11.9527)


![Image: GBG street types](GBG_street_types.png)


What I found interesting is that in the city centre, streets mostly end in "gatan" (red), while on the outskirts they mostly end in "vägen" (blue). This makes sense, since "vägen" means "road" and these are wider and have more heavy traffic than streets (gatan). However, "vägen" also means "way" which can also mean a small, calm street, almost a foot path. That's why in the suburbs in between houses and in smaller residential areas, the number of "vägen" is also very high.

This can be seen e.g. in the southern archipelago of Gothenburg, which consists of islands with cute red houses and streets that are almost devoid of car traffic (and most of them end in "vägen"). 

![Image: Island street types](GBG_islands_street_types.png)